In [2]:
from pypdb import Query
from tqdm.notebook import tqdm
from Bio.PDB import PDBParser, PDBIO


In [2]:
number_to_sequence = {}

In [3]:
with open("./data/TriZOD_test_set.fasta.txt", "r") as f:
    # number in first lines, sequence in second
    while True:
        number = f.readline()
        sequence = f.readline()
        if not sequence:
            break
        number_to_sequence[number.strip()[1:]] = sequence.strip()

In [4]:
def get_exact_pdb_match(seq):

    q = Query(seq, 
            query_type="sequence", 
            return_type="polymer_entity",
            )
    search_res = q.search()
    
    if search_res is None:
        return None

    only_1_match = [x for x in search_res["result_set"]
                    if x['services'][0]["nodes"][0]['match_context'][0]['sequence_identity'] == 1.0 and
                    x['services'][0]["nodes"][0]['match_context'][0]['alignment_length'] == len(seq)]
    
    if len(only_1_match) == 1:
        return only_1_match[0]
    else:
        return None

In [5]:
our_index_to_pdb = {}

for key in tqdm(number_to_sequence):
    pdb_match = get_exact_pdb_match(number_to_sequence[key])
    if pdb_match:
        our_index_to_pdb[key] = pdb_match

/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))


  0%|          | 0/348 [00:00<?, ?it/s]

/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/pypdb/util/http_requests.py:65: UserWarning: Too many failures on requests. Exiting...
  warnings.warn("Too many failures on requests. Exiting...")
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/pypdb/pypdb.py:303: UserWarning: Retrieval failed, returning None
  warnings.warn("Retrieval failed, returning None")


In [25]:
# from Bio.PDB import PDBList


# pdbl = PDBList()

# pdbl.download_pdb_files([x['identifier'].split('_')[0] for x in our_index_to_pdb.values()], pdir="./pdb_files", file_format="pdb")

Structure exists: './pdb_files/pdb4bxl.ent' 


In [6]:
from Bio.PDB import PDBParser, PDBIO


our_index_to_pdb_files = {}

three_to_one = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C', 
    'GLU': 'E', 'GLN': 'Q', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 
    'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P', 
    'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
    # Additional common amino acids
    'SEC': 'U', 'PYL': 'O', 
    # # Standard 3-letter codes for ambiguous amino acids
    # 'ASX': 'B', 'GLX': 'Z', 'XAA': 'X', 
    # # Selenocysteine and pyrrolysine
    # 'SEL': 'U', 'PYL': 'O', 
    # # Unknown or 'other' amino acid
    # 'XLE': 'J', 'UNK': 'X'
}

for k, v in tqdm(our_index_to_pdb.items()):
    prot_id, model_id = v['identifier'].split('_')
    #print(model_id)
    pdb_parser = PDBParser()
    structure = pdb_parser.get_structure(prot_id, f"pdb_files/pdb{str.lower(prot_id)}.ent")
    chains = [list(x.get_residues()) for x in structure.get_chains()]
    raw_chains = list(structure.get_chains())
    chains += [structure.get_residues()]
    
    # select such model that sequence is the same as for our protein
    
    our_seq = number_to_sequence[k]
    found = False
    for chain, raw_chain in zip(chains, raw_chains):
        residues_str = [three_to_one[x.get_resname()] for x in chain if x.get_resname() in three_to_one ]
        if "".join(residues_str) == our_seq:
            our_index_to_pdb_files[k] = raw_chain
            found = True
            break
        
    if not found:
        print("No chain found for protein", prot_id, k)

/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))


  0%|          | 0/161 [00:00<?, ?it/s]

/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))


No chain found for protein 2RPA 11075_1_1_1
No chain found for protein 2JZC 15617_1_1_1


/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3115.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5880.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8645.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11410.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 14175.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packag

...

No chain found for protein 4AIX 19671_1_1_1
No chain found for protein 2MQH 25033_1_1_1
No chain found for protein 6MNT 25263_1_1_1
No chain found for protein 2NBS 25994_1_1_1
No chain found for protein 4ZCE 26562_1_1_1


/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1853.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 1961.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6638.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 7065.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1394.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages

No chain found for protein 2L6K 17314_1_1_1
No chain found for protein 2L9L 17477_1_1_1
No chain found for protein 2LCS 17629_1_1_1


/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8778.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8828.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17154.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 17204.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 25530.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packa

No chain found for protein 2M38 17934_1_1_1
No chain found for protein 7X7S 18133_1_1_1
No chain found for protein 2M33 18919_1_1_1
No chain found for protein 2M4H 19003_1_1_1
No chain found for protein 2M7B 19179_1_1_1


In [7]:
# save all mapped proteins to file
for k, v in our_index_to_pdb_files.items():
    name = our_index_to_pdb.get(k)['identifier'].split('_')[0]
    io = PDBIO()
    io.set_structure(v)
    io.save(f"pdb_matched/{k};{name}.pdb")

In [10]:
import os

PDB_PATH = "pdb_matched"

with open("inputs", "w") as f:
    for file in os.listdir(PDB_PATH):
        f.write(os.path.realpath(os.path.join(PDB_PATH, file)) + "\n")

In [11]:
# write keys to single file
with open("pdb_matched/final_test_ids.txt", "w") as f:
    for k in our_index_to_pdb_files.keys():
        f.write(k + "\n")